# 🧩 L4 GPU 기준 추천 모델 구성

## 역할	모델 (파라미터 수)	이유

### 🔍 코드 변경 분석	Qwen2.5-Coder 3B	빠르고 정밀한 코드 diff 분석용. 24GB에서도 여유롭게 돌릴 수 있음
### 🧾 커밋 요약 & 태깅	phi-2 (2.7B) or TinyLlama (1.1B)	빠르게 메타 요약/태그 추출. Fast tokenizer 지원
### 📊 메타 데이터 관리	internal logic (Python logic)	LangGraph node에서 로컬 rule 기반 처리 가능
### ✍️ TIL 문장 작성	gemma 3-4b-it	인간스러운 문장력은 이걸로 담당

In [10]:
# !pip install -q -r "/content/drive/MyDrive/Github_moosunny/NLP_practice/AI Agent with RAG Systems/requirements.txt"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 40.2 MB/s eta 0:00:00


In [2]:
pip install langgraph datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver doe

In [1]:
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain.schema import SystemMessage, HumanMessage
from typing import TypedDict, List
from datasets import load_dataset, Dataset

In [4]:
dataset = load_dataset("ewhk9887/korean_code_reviews_from_github")
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

(…)rean_code_reviews_from_github_part1.json:   0%|          | 0.00/48.4M [00:00<?, ?B/s]

(…)rean_code_reviews_from_github_part2.json:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

(…)rean_code_reviews_from_github_part3.json:   0%|          | 0.00/16.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28054 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['code', 'lang', 'review'],
        num_rows: 28054
    })
})

In [9]:
dataset_jsonl = dataset['train'].to_json("train.jsonl", orient="records", lines=True)
dataset_jsonl

Creating json from Arrow format:   0%|          | 0/29 [00:00<?, ?ba/s]

89436459

In [12]:
with open("train.jsonl", "r") as f:
    for _ in range(5):
        print(f.readline())

{"code":"@@ -0,0 +1,32 @@\n+import * as S from '.\/Header.styled';\n+\n+interface HeaderProps extends React.InputHTMLAttributes<HTMLInputElement> {\n+  searchKeyword: string;\n+  onSearch: (keyword: string) => void;\n+}\n+\n+function LargeHeader({ searchKeyword, onSearch, ...rest }: HeaderProps) {\n+  return (\n+    <S.Layout>\n+      <a href=\"https:\/\/chosim-dvlpr.github.io\/react-movie-review\/\">\n+        <S.LogoButton>\n+          <S.LogoImage src=\".\/logo.png\" \/>\n+        <\/S.LogoButton>\n+      <\/a>\n+      <S.InputBox $isExpanded={true}>\n+        <S.Input\n+          {...rest}\n+          value={searchKeyword}\n+          onChange={(event: React.ChangeEvent<HTMLInputElement>) =>\n+            onSearch(event.target.value)\n+          }\n+          $isDisabled={false}\n+          disabled={false}\n+        \/>\n+        <S.SearchIcon src=\".\/search_button.png\" \/>\n+      <\/S.InputBox>\n+    <\/S.Layout>\n+  );\n+}\n+\n+export default LargeHeader;","lang":"Unknown","r

In [38]:
# jsonl 파일 불러오기
dataset_jsonl = load_dataset("json", data_files="/content/drive/MyDrive/KTB 파이널 프로젝트/github_code_review_unknown.jsonl", split = "train")
dataset_jsonl

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['code', 'lang', 'review'],
    num_rows: 3090
})

In [39]:
dataset_jsonl[0]

{'code': '@@ -0,0 +1,32 @@\n+import * as S from \'./Header.styled\';\n+\n+interface HeaderProps extends React.InputHTMLAttributes<HTMLInputElement> {\n+  searchKeyword: string;\n+  onSearch: (keyword: string) => void;\n+}\n+\n+function LargeHeader({ searchKeyword, onSearch, ...rest }: HeaderProps) {\n+  return (\n+    <S.Layout>\n+      <a href="https://chosim-dvlpr.github.io/react-movie-review/">\n+        <S.LogoButton>\n+          <S.LogoImage src="./logo.png" />\n+        </S.LogoButton>\n+      </a>\n+      <S.InputBox $isExpanded={true}>\n+        <S.Input\n+          {...rest}\n+          value={searchKeyword}\n+          onChange={(event: React.ChangeEvent<HTMLInputElement>) =>\n+            onSearch(event.target.value)\n+          }\n+          $isDisabled={false}\n+          disabled={false}\n+        />\n+        <S.SearchIcon src="./search_button.png" />\n+      </S.InputBox>\n+    </S.Layout>\n+  );\n+}\n+\n+export default LargeHeader;',
 'lang': 'Unknown',
 'review': '라우팅

In [29]:
unknown_list = []
lang_list = []

for json in dataset_jsonl:
  if json["lang"]  == "Unknown":
    unknown_list.append(json)
  else:
    lang_list.append(json)

In [33]:
lang_dataset = Dataset.from_list(lang_list)
unknown_dataset = Dataset.from_list(unknown_list)

unknown_jsonl = unknown_dataset.to_json("github_code_review_unknown.jsonl", orient="records", lines=True)
lang_jsonl = lang_dataset.to_json("github_code_review.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from google.colab import userdata

HUGGINGFACE_TOKEN = userdata.get('Hugging_Face_KEY')

In [5]:
code_model_name = "Qwen/Qwen2.5-Coder-3B-Instruct"
code_tokenizer = AutoTokenizer.from_pretrained(code_model_name)
code_model = AutoModelForCausalLM.from_pretrained(code_model_name, torch_dtype=torch.float16, device_map="auto")

til_model_name = "google/gemma-3-4b-it"
til_tokenizer = AutoTokenizer.from_pretrained(til_model_name, token = HUGGINGFACE_TOKEN, special_token = True)
til_model = AutoModelForCausalLM.from_pretrained(til_model_name, token = HUGGINGFACE_TOKEN,torch_dtype=torch.float16, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: Loading a multimodal model with `AutoModelForCausalLM` is deprecated and will be removed in v5. `AutoModelForCausalLM` will be used to load only the text-to-text generation module.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from PIL import Image
import requests

model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id, token = HUGGINGFACE_TOKEN)

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/bee.jpg"},
            {"type": "text", "text": "Describe this image in detail."}
        ]
    }
]

inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Okay, here's a detailed description of the image:

**Overall Impression:**

The image is a close-up shot of a vibrant garden scene, focusing on a cluster of pink cosmos flowers and a busy bumblebee. It has a slightly soft, natural feel, likely captured in daylight.

**Foreground:**

*   **Cosmos Flowers:** The dominant feature is a large, bright pink cosmos flower in the center. It’s fully open, displaying its layered petals and a yellow


In [52]:
from typing import Annotated, List, TypedDict, Literal
from langgraph.graph.message import add_messages

# ----------------------
# 상태 정의
# ----------------------
class TILState(TypedDict):
  commit_summary: str # 커밋 내용 요약
  problem: str # 겪은 문제 (커밋 메시지)
  approach: str # 접근 방법 (코드 내용)
  solution: str # 해결한 방법 (코드 내용)
  code_snippet: str
  til_markdown: str

In [ ]:
def analyze_code_diff(state: TILState) -> TILState:
    # 여기서는 Qwen2.5 Coder 등을 호출하여 코드 변경 요약한다고 가정
    code_diff = """@@ -1,5 +1,8 @@\n+ Added Transformer training loop\n+ Added BLEU score calculation\n"""
    summary = "Transformer 학습 루프 및 BLEU Score 계산 모듈 추가"
    return {**state, "diff_summary": summary}

In [ ]:
def generate_meta_tags(state: TILState) -> TILState:
    # phi-2 또는 TinyLlama 같은 모델이 역할
    summary = state["diff_summary"]
    tags = "#Transformer #BLEU #Training"
    return {**state, "meta_tags": tags}

In [ ]:
def write_til_entry(state: TILState) -> TILState:
    # GPT-4o 호출하는 부분 (예: OpenAI API)
    system = SystemMessage(content="당신은 개발자의 일일 학습 로그를 요약하는 어시스턴트입니다.")
    human = HumanMessage(content=f"오늘 작업 요약: {state['diff_summary']}\n관련 태그: {state['meta_tags']}\n위 내용을 바탕으로 TIL 마크다운 문장을 작성해줘.")

    # 아래는 예시 응답. 실제 GPT 호출로 대체 가능
    til_text = "## TIL\n- Transformer 학습 루프를 구현하고 BLEU Score 계산을 위한 유틸리티를 추가했다.\n- 학습 손실 시각화 및 퍼플렉서티 계산 로직도 실험해봤다."
    return {**state, "til_text": til_text}

In [ ]:
# ----------------------
# LangGraph 정의
# ----------------------
graph = StateGraph(TILState)

# 노드 등록
graph.add_node("analyze_diff", analyze_code_diff)
graph.add_node("generate_tags", generate_meta_tags)
graph.add_node("write_til", write_til_entry)

# 엣지 연결
graph.set_entry_point("analyze_diff")
graph.add_edge("analyze_diff", "generate_tags")
graph.add_edge("generate_tags", "write_til")
graph.add_edge("write_til", END)

# 컴파일
til_app = graph.compile()

In [ ]:
# ----------------------
# 실행 예시
# ----------------------
if __name__ == "__main__":
    initial_state = {}
    result = til_app.invoke(initial_state)
    print("\n--- 생성된 TIL ---")
    print(result["til_text"])